# AUTOMATIC TRENDLINES DETECTION

In [ ]:
!git clone https://github.com/rfoucault/3icapital.git
%cd 3icapital

In [1]:
import numpy as np
import pandas as pd
import warnings
from functions import plot_candles, get_best_fit_slope_intercept, fit_trendlines, compute_trendline_scores_parallel, candles_close_to_trendline_numba, dist_from_trendline_numba, compute_canal_score, is_wedge

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')


## 1. Choose a stock 

In [2]:
data = pd.read_csv('data/BTC.csv')

In [3]:
data.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,symbol
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17 04:59:59,202366.138393,171,35.160503,150952.477943,BTCUSDT
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17 05:59:59,100304.823567,102,21.448071,92608.279728,BTCUSDT
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17 06:59:59,31282.312670,36,4.802861,20795.317224,BTCUSDT
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17 07:59:59,19241.058300,25,2.602292,11291.347015,BTCUSDT
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 08:59:59,4239.503586,28,0.814655,3552.746817,BTCUSDT


- Choose an interval, recommended maximum 4 days.
- With an hour timeframe, 4 days get nearly 100 candles. Better for computations 

In [4]:
start = '2025-02-18'
end =  '2025-02-22'

In [5]:
data['date'] = data['open_time'].astype('datetime64[s]')
data = data.set_index('date')

data = data[['open', 'high', 'low', 'close']]
df = data[(data.index > start) & (data.index < end)]
df['time'] = pd.to_datetime(df.index)
print(f"candles number : {df.shape[0]}")

candles number : 95


In [6]:
df.head()

,open,high,low,close,time
date,,,,,
2025-02-18 01:00:00,95637.93,96300.00,95606.46,96266.24,2025-02-18 01:00:00
2025-02-18 02:00:00,96266.24,96322.97,96015.00,96064.47,2025-02-18 02:00:00
2025-02-18 03:00:00,96064.47,96290.00,96000.00,96200.00,2025-02-18 03:00:00
2025-02-18 04:00:00,96200.00,96300.73,95968.10,96070.81,2025-02-18 04:00:00
2025-02-18 05:00:00,96070.81,96110.63,95539.14,95609.49,2025-02-18 05:00:00


In [7]:
fig = plot_candles(df)

---
## 2. Find the best Support and resist on all your data 

- Find best Linear Fit
- Shift down best linear fit, to be below all candles (adjust intercept)
- Solve constrained optimization problem (adjust slope and intercept)
    - Least squares fit
    - Line below all candles

### STEP 1 : Simple Linear Regression 

In [8]:
X = np.arange(len(df))
y = df['close']

In [9]:
slope_0, intercept_0 = get_best_fit_slope_intercept(y)
init_trendline = slope_0 * X + intercept_0

In [10]:
fig = plot_candles(df, trendlines=[
    {
        'slope': slope_0,
        'intercept': intercept_0,
        'start_time': df.index[0],
        'end_time': df.index[-1],
        'name': 'Support',
        'color': 'blue'
    },
])

### STEP 2 : Shift the line below the minimum pivot and upper the maximum pivot

In [11]:
upper_pivot = (y - init_trendline).argmax() 
lower_pivot = (y - init_trendline).argmin() 

intercept_resistance_0 = -slope_0 * upper_pivot + y[upper_pivot]
intercept_support_0 = -slope_0 * lower_pivot + y[lower_pivot]

In [12]:
fig = plot_candles(df, trendlines=[
    {
        'slope': slope_0,
        'intercept': intercept_resistance_0,
        'start_time': df.index[0],
        'end_time': df.index[-1],
        'name': 'Support',
        'color': 'blue'
    },
        {
        'slope': slope_0,
        'intercept': intercept_support_0,
        'start_time': df.index[0],
        'end_time': df.index[-1],
        'name': 'Support',
        'color': 'blue'
    },
])

### STEP 3: Do Constrained Optimization

🔁 fit_trendlines : Finds the best slope that minimizes the SSE, while making sure all prices stay below (for resistance) or above (for support) the trendline : constraint optimization
- This function fine-tunes the angle of each trendline
    - It tries slightly different slopes to see which one fits best with the data, comparing the SSE
    - It picks the one that keeps the line under (or over) the prices while staying as close as possible



In [13]:
y_numpy = y.to_numpy()

In [14]:
support_1_coefs, resistance_1_coefs = fit_trendlines(y_numpy)

In [15]:
fig = plot_candles(df, trendlines=[
    {
        'slope': support_1_coefs[0],
        'intercept': support_1_coefs[1],
        'start_time': df.index[0],
        'end_time': df.index[-1],
        'name': 'Support',
        'color': 'blue'
    },
        {
        'slope': resistance_1_coefs[0],
        'intercept': resistance_1_coefs[1],
        'start_time': df.index[0],
        'end_time': df.index[-1],
        'name': 'resistance',
        'color': 'red'
    },
])

### Is these good trendlines ?

🕯️ candles_close_to_trendline(...)
This function checks how many candles (price points) are close to a trendline.

🔍 How it works:
- Calculates the distance between each price and the trendline
- If the distance is smaller than the typical candle size (median_candle_width), it counts it as "close"

Works differently for:
- Support lines → prices should be above the line
- Resistance lines → prices should be below the line

In [18]:
percentile = 50

In [19]:
median_candle_width = np.nanpercentile(np.abs(df['open'] - df['close']), percentile)
print('the median size of a candle :',median_candle_width)

the median size of a candle : 201.77000000000407


In [20]:
print('number of candles close to the support :',candles_close_to_trendline_numba(y_numpy, support_1_coefs[0], support_1_coefs[1], median_candle_width, support=True), '| number of candles close to the resist :', candles_close_to_trendline_numba(y_numpy, resistance_1_coefs[0], resistance_1_coefs[1], median_candle_width, support=False))

number of candles close to the support : 4 | number of candles close to the resist : 3


📏 dist_from_trendline(...)
This function measures how far the prices are from the trendline.

🔍 What it does:
- Calculates the distance between each price and the trendline.
- Then returns the 50 percentile of those distances.

This gives a sense of the worst-case gap — how far most prices are from the line.

⚙️ Why it matters:
- A lower distance means the trendline follows prices closely (better fit).
- Helps filter out lines that are too far from actual price action.

✅ Used to compare and score trendlines based on how well they "hug" the price.

In [21]:
dist_from_trendline_numba(y_numpy, support_1_coefs[0], support_1_coefs[1], percentile, support=True), dist_from_trendline_numba(y_numpy, resistance_1_coefs[0], resistance_1_coefs[1], percentile, support=False)

(2272.8318428242783, 1204.7753070966282)

---

## 3: Repeat the process over all possible windows to find the best overall trendlines

### STEP 1 :🔁 Scanning All Windows for the Best Trendlines

To find the most reliable support and resistance lines, we apply the detection process on all possible time windows in the dataset.

🧮 compute_trendline_scores(...)
This function slides across the dataset, testing every possible sub-window (of at least min_window candles), and for each one:

- Computes the best support and resistance trendlines
- Evaluates how many candles are close to each line
- Measures how well the line fits (distance from price)
- Records the slope, intercept, and stats in a score table

Each window becomes a candidate, and we can later sort and filter them to find the most accurate trendlines.

In [22]:
df_score = compute_trendline_scores_parallel(df, percentile)

Analyse en parallèle: 100%|██████████| 2850/2850 [00:13<00:00, 204.81it/s]


In [23]:
df_score.head()

,support_line,resistance_line,candles_close_to_trendline,dist_from_trendline,range,candles_number
0,"{'slope': -121.55521627054588, 'intercept': 95...","{'slope': -41.0771464347012, 'intercept': 9672...","{'support': 7, 'resistance': 2}","{'support': 452.64175594180415, 'resistance': ...","{'start': 2025-02-18 01:00:00, 'end': 2025-02-...",21
1,"{'slope': -121.55771811469693, 'intercept': 95...","{'slope': -46.42723599244885, 'intercept': 967...","{'support': 7, 'resistance': 2}","{'support': 452.6042282795388, 'resistance': 6...","{'start': 2025-02-18 01:00:00, 'end': 2025-02-...",22
2,"{'slope': -121.55856505141442, 'intercept': 95...","{'slope': -55.83556769450186, 'intercept': 969...","{'support': 7, 'resistance': 3}","{'support': 472.83286989716, 'resistance': 701...","{'start': 2025-02-18 01:00:00, 'end': 2025-02-...",23
3,"{'slope': -121.55803281789603, 'intercept': 95...","{'slope': -55.83348157888219, 'intercept': 969...","{'support': 7, 'resistance': 3}","{'support': 472.8339343642001, 'resistance': 6...","{'start': 2025-02-18 01:00:00, 'end': 2025-02-...",24
4,"{'slope': -121.55199338191123, 'intercept': 95...","{'slope': -52.22349230769229, 'intercept': 968...","{'support': 7, 'resistance': 4}","{'support': 515.0939867638372, 'resistance': 6...","{'start': 2025-02-18 01:00:00, 'end': 2025-02-...",25


### STEP 2 : Find the last best trendlines

🧮 Canal Scoring Function
We use a custom score to evaluate how "good" a price channel is (a pair of support + resistance lines). The goal is to favor channels that are:

- Well aligned with price (small distance to the lines)
- Respected by price action (many candles touch the lines)
- Spread over time (longer duration = more meaningful)

⚙️ compute_canal_score(...)
This function calculates a score based on:

- 🧲 Proximity → how close prices are to the trendlines
- 🔁 Touches → number of candles near each line
- ⏳ Duration → how long the trend lasted (in hours)

score = (
    w_proximity * avg_proximity +
    w_bougies * touch_score +
    w_duration * duration
)

🔸 w_proximity, w_bougies, and w_duration are weights you can adjust depending on what matters most to you.

In [24]:
df_score['canal_score'] = df_score.apply(
    lambda row: compute_canal_score(row, w_duration=0.03, w_proximity=1, w_bougies=0.5),
    axis=1
)

# Trier et récupérer la meilleure ligne
best_row = df_score.sort_values(by='canal_score', ascending=False).iloc[0]
best_row[['support_line', 'resistance_line', 'range', 'canal_score']].to_dict()


{'support_line': {'slope': 66.2106460107908, 'intercept': 92864.65966382735},
 'resistance_line': {'slope': 34.7313434587859,
  'intercept': 96165.26865654122},
 'range': {'start': Timestamp('2025-02-18 02:00:00'),
  'end': Timestamp('2025-02-21 12:00:00')},
 'canal_score': 5.711166077858318}

In [25]:
fig_ = plot_candles(df, trendlines=[
    {
        'slope': best_row['support_line']['slope'],
        'intercept': best_row['support_line']['intercept'],
        'start_time': best_row['range']['start'],
        'end_time': best_row['range']['end'],
        'name': 'Support',
        'color': 'blue'
    },
        {
        'slope': best_row['resistance_line']['slope'],
        'intercept': best_row['resistance_line']['intercept'],
        'start_time': best_row['range']['start'],
        'end_time': best_row['range']['end'],
        'name': 'resistance',
        'color': 'red'
    },
])

In [26]:
is_wedge(
    support_line={'slope': best_row['support_line']['slope'], 'intercept': best_row['support_line']['intercept']},
    resistance_line={'slope': best_row['resistance_line']['slope'], 'intercept': best_row['resistance_line']['intercept']},
    range_window={'start': best_row['range']['start'], 'end': best_row['range']['end']}
)


True

## The two trends form a rising channel that’s tightening, known as a rising (ascending) wedge. More in-depth analysis can be carried out to test a trading strategy based on these signals…